In [7]:
import geneformer
from geneformer import TranscriptomeTokenizer
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import loompy
from collections import Counter

In [8]:
# Have to redo tokenization while permutating the data to see evaluate geneformer model
# Tk will tokenize all loom files in a directory so make sure each loom file as its own directory
data_path = "/u/home/r/ramadas/geneformer_tokenized_data"
output_path = "/u/home/r/ramadas/geneformer_tokenized_data/permuted"

In [9]:
import os
import loompy
import numpy as np
from collections import Counter

# Org file 
original_loom = "/u/home/r/ramadas/geneformer_tokenized_data/labeled_t_cell_data.loom"
# Copy so we can permute it
permuted_loom = "/u/home/r/ramadas/geneformer_tokenized_data/labeled_t_cell_data_permuted.loom"

# Remove the file if it already exists
if os.path.exists(permuted_loom):
    os.remove(permuted_loom)
    
with loompy.new(permuted_loom) as ds_permute:
    with loompy.connect(original_loom) as ds:
        ds_permute.add_columns(ds.layers, col_attrs=ds.col_attrs, row_attrs=ds.row_attrs)

        original_stim = list(ds.ca['stimulation'])
        print("Original stimulation distribution:", Counter(original_stim))

        print("First 10 original values:", original_stim[:10])
        
        # Create a copy and shuffle it
        permuted_stim = np.array(original_stim, dtype=np.str_).copy()
        np.random.shuffle(permuted_stim)  # Shuffles in-place

        print("First 10 shuffled values:", permuted_stim[:10])

        same_position = np.mean(original_stim == permuted_stim) * 100
        print(f"Percentage of values in same position: {same_position:.2f}%")
        
        # Replace the stimulation data in the new file
        ds_permute.ca['stimulation'] = permuted_stim
        print("Updated permuted file successfully")




Original stimulation distribution: Counter({np.str_('act'): 24249, np.str_('rest'): 23477})
First 10 original values: [np.str_('rest'), np.str_('act'), np.str_('rest'), np.str_('rest'), np.str_('rest'), np.str_('act'), np.str_('rest'), np.str_('rest'), np.str_('act'), np.str_('rest')]
First 10 shuffled values: ['rest' 'rest' 'act' 'act' 'act' 'rest' 'rest' 'act' 'rest' 'act']
Percentage of values in same position: 50.13%
Updated permuted file successfully


In [10]:
# Geneformer created new models so we have to specify 
tk = TranscriptomeTokenizer({'stimulation': 'stimulation'}, nproc=16, model_input_size = 2048)
tk.tokenize_data(data_path, output_path, "Q_data_permuted", file_format="loom")


Tokenizing /u/home/r/ramadas/geneformer_tokenized_data/labeled_t_cell_data.loom


100% 94/94 [01:36<00:00,  1.02s/it]


/u/home/r/ramadas/geneformer_tokenized_data/labeled_t_cell_data__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /u/home/r/ramadas/geneformer_tokenized_data/labeled_t_cell_data_permuted.loom


100% 94/94 [01:28<00:00,  1.06it/s]


/u/home/r/ramadas/geneformer_tokenized_data/labeled_t_cell_data_permuted__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.
